In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pandas_datareader as pdr
import talib as tl

In [3]:
from sklearn.metrics import confusion_matrix, accuracy_score
def evaluate_model(dt_classifier):
    print("Train Accuracy :", accuracy_score(y_train, dt_classifier.predict(X_train)))
    print("Train Confusion Matrix:")
    print(confusion_matrix(y_train, dt_classifier.predict(X_train)))
    print("-"*50)
    print("Test Accuracy :", accuracy_score(y_test, dt_classifier.predict(X_test)))
    print("Test Confusion Matrix:")
    print(confusion_matrix(y_test, dt_classifier.predict(X_test)))

In [4]:
#Read csv file 
#Dataset A
HSI_4y = pd.read_csv('../dataset/^HSI_4y.csv')
N225_4y = pd.read_csv('../dataset/^N225_4y.csv')
SSE_4y = pd.read_csv('../dataset/000001.SS_4y.csv')
DJI_4y = pdr.get_data_yahoo("^DJI", start="2012-01-01", end="2016-12-31").reset_index() #Nikkei 225

#Dataset B
HSI_7y = pd.read_csv('../dataset/^HSI_7y.csv')
N225_7y = pd.read_csv('../dataset/^N225_7y.csv')
SSE_7y = pd.read_csv('../dataset/000001.SS_7y.csv')
DJI_7y = pdr.get_data_yahoo("^DJI", start="2012-01-01", end="2019-12-31").reset_index()#Nikkei 225

In [5]:
#chose the data
df = DJI_4y
df = df.drop(columns=['Date']).dropna()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1258 entries, 0 to 1257
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   High       1258 non-null   float64
 1   Low        1258 non-null   float64
 2   Open       1258 non-null   float64
 3   Close      1258 non-null   float64
 4   Volume     1258 non-null   int64  
 5   Adj Close  1258 non-null   float64
dtypes: float64(5), int64(1)
memory usage: 68.8 KB


In [6]:
#add Technical indicator
close = df['Close'].values
high = df['High'].values
low = df['Low'].values
volume = df['Volume'].values.astype(float)
#Simple movingaverage (SMA)
df['SMA5'] = tl.SMA(close, timeperiod=5)
df['SMA10'] = tl.SMA(close, timeperiod=10)
df['SMA14'] = tl.SMA(close, timeperiod=14)
df['SMA30'] = tl.SMA(close, timeperiod=30)

#Exponentialmoving average
df['EMA5'] = tl.EMA(close, timeperiod=5)
df['EMA10'] = tl.EMA(close, timeperiod=10)
df['EMA14'] = tl.EMA(close, timeperiod=14)
df['EMA30'] = tl.EMA(close, timeperiod=30)

#Momentum indicator
df['MOM5'] = tl.MOM(close, timeperiod=5)
df['MOM10'] = tl.MOM(close, timeperiod=10)
df['MOM14'] = tl.MOM(close, timeperiod=14)

#Moving average convergence divergence
df['MACD'], df['MACDSIGNAL'], df['macdhist'] = tl.MACD(close, fastperiod=12, slowperiod=26, signalperiod=9)

#Stochastic oscillator
df['STOCHK'], df['STOCHD'] = tl.STOCH(high, low, close, fastk_period=5, slowk_period=3, slowk_matype=0, slowd_period=3, slowd_matype=0)

#Relative strength index
df['RSI14'] = tl.RSI(close, timeperiod=14)
df['RSI28'] = tl.RSI(close, timeperiod=28)

#Williams R
df['WILLR14'] = tl.WILLR(high, low, close, timeperiod=14)
df['WILLR28'] = tl.WILLR(high, low, close, timeperiod=28)

#Accumulation distribution index
df['AD']= tl.AD(high, low, close, volume)


#Commodity channel index
df['CCI14'] = tl.CCI(high, low, close, timeperiod=14)    
df['CCI28'] = tl.CCI(high, low, close, timeperiod=28)    

#Rate of change 
df['ROC'] = tl.ROC(close, timeperiod=2)

#Average True Range
df['ATR'] = tl.ATR(high, low, close, timeperiod=14)

df = df.dropna()

In [8]:
#Create the Target column
df['Target'] = np.where(df['Close'].shift(-1) > df['Close'], 1, 0)
df['Target2'] = np.where(100*((df['Close'].shift(-1)-df['Open'])/df['Open']) > 0 , 1, 0)

In [10]:
#Split the data set into the a feature or indipendent data set (X) and a Target or dependant data set (Y)
keep_columns = df.drop(['High','Low','Open','Volume','Adj Close','macdhist','Target','Target2'],axis=1).columns
X = df[keep_columns].values
y = df['Target2'].values
print(X.shape)
print(y.shape)

(1225, 25)
(1225,)


In [14]:
#Scale the data

#StandardScaler
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X)
X_scaled = scaler.transform(X)


In [15]:
from sklearn.linear_model import LogisticRegression
estimator = LogisticRegression()

In [16]:
#Feature selection
#use RFE
from sklearn.feature_selection import RFE
rfe = RFE(estimator, n_features_to_select=5, step=1)
rfe = rfe.fit(X, y)
X_new = rfe.fit_transform(X_scaled, y)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

In [17]:
#Split test train data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_new, y, test_size = 0.15, shuffle=False)

In [19]:
#Hyperparameter tuning
from sklearn.model_selection import GridSearchCV

# Create the parameter grid based on the results of random search 
params = {
    'solver': ['newton-cg', 'lbfgs', 'liblinear'],
    'penalty': ['l2'],
    'C': [100, 10, 1.0, 0.1, 0.01]
}

# Instantiate the grid search model
grid_search = GridSearchCV(estimator=estimator, 
                           param_grid=params, 
                           cv=4, n_jobs=-1, verbose=1, scoring = "accuracy")

print(grid_search.fit(X_train, y_train))
print(grid_search.best_estimator_)
rf_best = grid_search.best_estimator_

Fitting 4 folds for each of 15 candidates, totalling 60 fits
GridSearchCV(cv=4, estimator=LogisticRegression(), n_jobs=-1,
             param_grid={'C': [100, 10, 1.0, 0.1, 0.01], 'penalty': ['l2'],
                         'solver': ['newton-cg', 'lbfgs', 'liblinear']},
             scoring='accuracy', verbose=1)
LogisticRegression(C=100, solver='newton-cg')


In [20]:
evaluate_model(rf_best)

Train Accuracy : 0.6974063400576369
Train Confusion Matrix:
[[244 206]
 [109 482]]
--------------------------------------------------
Test Accuracy : 0.6847826086956522
Test Confusion Matrix:
[[44 41]
 [17 82]]
